In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

# TFRcord概述
     tfrecord 
     -> tf.train.Example
       -> tf.train.Features -> {"key": tf.train.Feature}
          -> tf.train.Feature -> tf.train.ByteList/FloatList/Int64List
          
    tfrecord是tf中定义的一种文件格式，因为经过tf的优化，所以读取速度更快。
    tfrecord文件由tf.train.Example组成，每个Example可以是一个元素，也可以是一组元素。
    Example又由tf.train.Feature组成，
    feature是格式类似与字典的数据，字典的key代表数据的含义，value代表数据的值，这些值为ByteList，Int64List，Float32List类型。    
    
# 生成value相关函数：
    tf.train.ByteList(value):对字符串类型的数据进行转换
    tf.train.FloatList(value):对浮点型的数据进行转换
    tf.train.Int64List(value):对整数型的数据进行转换
    
# 生成Feature/Features:
    tf.train.Feature()
        func:生成feature
        paras：ByteList = 相应的源数据
               FloatList = 
               Int64List = 
                 
    tf.train.Features(feature):整合feature为features
# 生成Example
    tf.train.example(feature):convert feature to example


In [ ]:
# 创建tfrecord文件
favorite_books = [name.encode('utf-8')
                  for name in ["machine learning", "cc150"]] #通过列表生成式将两个书名编码为'utf-8'
favorite_books_bytelist = tf.train.BytesList(value = favorite_books) 
# 通过tf.train.ByteList将utf8格式的数据转化为tf.train.ByteList
print(favorite_books_bytelist)

hours_floatlist = tf.train.FloatList(value = [15.5, 9.5, 7.0, 8.0])
print(hours_floatlist)


age_int64list = tf.train.Int64List(value = [42])
print(age_int64list)

features = tf.train.Features(
    feature = {
        "favorite_books": tf.train.Feature(
            bytes_list = favorite_books_bytelist),
        "hours": tf.train.Feature(
            float_list = hours_floatlist),
        "age": tf.train.Feature(int64_list = age_int64list),
    }
)
print(features)

# Example.SerializedToString()方法
    func：将Example序列化，方便存储与传输


In [ ]:
example = tf.train.Example(features=features)  # 生成Example
print(example)

# example.SerializeToString()将数据序列化，方便存储与传输
serialized_example = example.SerializeToString()
print(serialized_example)

# tf.io.TFRecordOptions()
    https://www.tensorflow.org/api_docs/python/tf/io/TFRecordOptions
    func:支持关于tfrecord的各种操作
    paras:compression_type:压缩类型，"GZIP", "ZLIB", or "" (no compression).
          其他压缩类型看官方文档。


# tf.io.TFRecordWriter(path, options)
    func:写入数据，生成tfrecord类型数据
    paras：path：写入文件的路径/文件名
           options：指定文件压缩类型，或者指定TFRecordOptions中其他操作。

In [ ]:
# 生成tfrecord文件   
output_dir = 'tfrecord_basic'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
filename = "test.tfrecords"
filename_fullpath = os.path.join(output_dir, filename)
# 通过专用的TFRecordWriter()写入，生成一个tfrecord文件
with tf.io.TFRecordWriter(filename_fullpath) as writer:
    for i in range(3):
        writer.write(serialized_example)


In [ ]:
dataset = tf.data.TFRecordDataset([filename_fullpath])
for serialized_example_tensor in dataset:
    print(serialized_example_tensor)

In [ ]:
# 通过tf.data.TFRecord()由文件生成序列化(serialized)数据集

expected_features = {
    "favorite_books": tf.io.VarLenFeature(dtype = tf.string),
    "hours": tf.io.VarLenFeature(dtype = tf.float32),
    "age": tf.io.FixedLenFeature([], dtype = tf.int64),
}
dataset = tf.data.TFRecordDataset([filename_fullpath])  # 生成序列化数据集，其中包含所有example
for serialized_example_tensor in dataset:  # 可以从record中分别取出example
    example = tf.io.parse_single_example(  # 对example进行unserialized，生成parse_tensor数据
        serialized_example_tensor,
        expected_features)  # expected_features的作用类似于decode_csv中的record_defaults
    books = tf.sparse.to_dense(example["favorite_books"],  # 通过tf.sparse.to_dense将某个key对应的value
                               default_value=b"")
    for book in books:
        print(book.numpy().decode("UTF-8"))

In [ ]:
# 将文件进行压缩
filename_fullpath_zip = filename_fullpath + '.zip'  # 在文件名后加上.zip后缀
options = tf.io.TFRecordOptions(compression_type = "GZIP")  
# tf.io.TFRecordOptions():对tfrecord文件进行一系列操作
with tf.io.TFRecordWriter(filename_fullpath_zip, options) as writer:
    for i in range(3):
        writer.write(serialized_example)
'''
tf.io.TFRecordWriter(path, options)
    func:写入数据，生成tfrecord类型数据
    paras：path：写入文件的路径/文件名
           options：指定文件压缩类型，或者指定TFRecordOptions中其他操作。

'''

# tf.data.TFRecordDataset()
    func:由tfrecord文件生成数据集
    paras：filenames：源文件名
           compression_type:压缩格式
           buffer_size:读入缓存的大小，以字节为单位。
           num_parallel_reads:并行读取文件的数量。

In [ ]:
# 将压缩文件解压
dataset_zip = tf.data.TFRecordDataset([filename_fullpath_zip], 
                                      compression_type= "GZIP")
'''
tf.data.TFRecordDataset()
    func:由tfrecord文件生成数据集
    paras：filenames：源文件名
           compression_type:压缩格式
           buffer_size:读入缓存的大小，以字节为单位。
           num_parallel_reads:并行读取文件的数量。
'''
for serialized_example_tensor in dataset_zip:
    example = tf.io.parse_single_example(
        serialized_example_tensor,
        expected_features)
    books = tf.sparse.to_dense(example["favorite_books"],
                               )  # 教程里要加上default_value=b""， 但是不加也可以
    for book in books:
        print(book.numpy().decode("UTF-8")) #最开始encode，最后要decode